In [ ]:
# Convert saved model to ONNX format

!python -m tf2onnx.convert --saved-model saved-model --output saved_model.onnx

In [ ]:
import numpy as np
import onnx
import onnxruntime as ort

In [ ]:
model = ort.InferenceSession('saved_model.onnx', providers=['CPUExecutionProvider'])

In [ ]:
input_name = model.get_inputs()[0].name
print("Input name  :", input_name)

input_name = model.get_inputs()[1].name
print("Input name  :", input_name)

output_name = model.get_outputs()[0].name
print("Output name :", output_name)

In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
file_path = 'combined-dataset/final_reviews_data.csv'
data = pd.read_csv(file_path)

# Encode the 'types' column
label_encoder = LabelEncoder()
data['types_encoded'] = label_encoder.fit_transform(data['types'])

# Tokenize the 'review' column
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(data['review'])

# Pad the sequences
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Create the feature set
X = {
    'review': padded_sequences,
    'types': data['types_encoded'].values,
}

# Normalize the sentiment scores
y = data['sentiment'].values


In [ ]:
review_input = X['review'].astype(np.float32)
types_input = np.expand_dims(X['types'].astype(np.float32), axis=1)

In [ ]:
input_data = {
    'review': review_input,
    'types': types_input,
}

In [ ]:
result = model.run(None, input_data)